## Step 1. Import necessary libraries

In [ ]:
# import itertools
import numpy as np
import pandas as pd
import seaborn as sns
import time

import matplotlib.pyplot as plt

from joblib import dump, load
from sklearn import preprocessing
from sklearn.compose import ColumnTransformer
from sklearn.decomposition import PCA
from sklearn.experimental import enable_halving_search_cv
from sklearn.feature_selection import chi2, SelectKBest
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay, multilabel_confusion_matrix
from sklearn.metrics import classification_report, f1_score, precision_score, recall_score
from sklearn.model_selection import cross_val_score, learning_curve, LearningCurveDisplay, train_test_split, validation_curve, ValidationCurveDisplay
from sklearn.model_selection import GridSearchCV, GroupKFold, HalvingGridSearchCV, RandomizedSearchCV
from sklearn.neural_network import MLPClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, Normalizer, OneHotEncoder, RobustScaler, StandardScaler
from sklearn.svm import SVC

from matplotlib import rcParams
rcParams.update({'figure.autolayout': True})

In [ ]:
# """
# plotting learning curve
# """

# fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 6), sharey=True)

# common_params = {
#     "X": X,
#     "y": y,
#     "train_sizes": np.linspace(0.1, 1.0, 5),
#     "cv": GroupKFold(n_splits=df[col_grp].nunique(), test_size=0.2, random_state=0),
#     "score_type": "both",
#     "n_jobs": 4,
#     "line_kw": {"marker": "o"},
#     "std_display_style": "fill_between",
#     "score_name": "Accuracy",
# }

# for ax_idx, estimator in enumerate([naive_bayes, svc]):
#     LearningCurveDisplay.from_estimator(estimator, **common_params, ax=ax[ax_idx])
#     handles, label = ax[ax_idx].get_legend_handles_labels()
#     ax[ax_idx].legend(handles[:2], ["Training Score", "Test Score"])
#     ax[ax_idx].set_title(f"Learning Curve for {estimator.__class__.__name__}")

## Step 2a. Declare user variables

In [ ]:
is_test = False

user_specified_ntbk_identifier = 'failure analysis' #to uniquely identify the results generated by this notebook
user_specified_output_folder = 'Failure Analysis/'

user_specified_layout = 'layout1'
user_specified_layout_int = int(user_specified_layout[len(user_specified_layout) - 1])
user_specified_nth = 3

user_test_size = 0.2
user_random_state = 1

if user_specified_layout_int == 1:
    ith_inp_col = 16
else:
    ith_inp_col = 20

n_steps = 2
list_filenames = {
    'layout1':'dataset_1.csv',
    'layout2':'dataset_2.csv',
}
list_columns = {
    'layout1_all':['timestamp','posture_id','posture_label','s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','birth_year','sex','height','weight','bmi','bmi_label','full_name','nth','round'],
    'layout2_all':['timestamp','posture_id','posture_label','s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20','birth_year','sex','height','weight','bmi','bmi_label','full_name','nth','round'],
    'layout1_cat_inp':[],
    'layout1_num_inp':['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16', 'height','weight','bmi'],
    'layout2_cat_inp':[],
    'layout2_num_inp':['s01','s02','s03','s04','s05','s06','s07','s08','s09','s10','s11','s12','s13','s14','s15','s16','s17','s18','s19','s20', 'height','weight','bmi'],
}
list_positions = ['Yearner_Right', 'Yearner_Left', 'Fetal_Right', 'Fetal_Left', 'Log_Right', 'Log_Left', 'Supine', 'Prone', # 'Empty'
]
dict_positions = {
    1   : 'Yearner_Right',
    2   : 'Yearner_Left',
    3   : 'Fetal_Right',
    4   : 'Fetal_Left',
    5   : 'Log_Right',
    6   : 'Log_Left',
    7   : 'Supine',
    8   : 'Prone',
    100 : 'YRLR',
    200 : 'YLLL',
}
list_grouped_positions = [
    'Fetal_Right',
    'Fetal_Left',
    'Supine',
    'Prone',
    'YRLR',
    'YLLL',
]

participants = {
    22.89 : 'Karen_Alli',
    23.73 : 'Jenalene_Ambat',
    25.34 : 'Rodrigo_Baldos',
    34.34 : 'Karlo_Barba',
    27.4  : 'Harold_Paulo_Bautista',
    29.41 : 'Gerwin_Rommel_Caballes',
    23.78 : 'Randy_Caldo',
    27.68 : 'Joey_Callos',
    23.83 : 'Delford_Gamayo',
    35.2  : 'Criselda_Garcia',
    24.31 : 'Hans_Oswald_Ibrahim',
    33.02 : 'Marven_Ignacio',
    19.35 : 'Zarah_Jane_Jolejole',
    26.24 : 'Karl_Michael_Leyson',
    24.91 : 'Ryan_Dominic_Lin',
    24.58 : 'Joanna_May_Lisondra',
    20.2  : 'Jose_Ignacio_Locsin',
    25.12 : 'Christopher_Madrona',
    27.02 : 'Karl_Justin_Ngu',
    25.01 : 'Gabriel_Joseph_Pua',
    24.61 : 'Catalina_Quiogue',
    24.22 : 'Michelle_Ramos',
    34.56 : 'Oliver_Reyes',
    25.4  : 'Emily_Robrigado',
    19.39 : 'Lloyd_Saavedra',
    42.52 : 'Mark_Gavin_Sanchez',
    20.34 : 'Flory-Anne_Sinco',
    23.53 : 'Adrian_Aerol_Supranes',
    28.62 : 'Vince_Allen_Sy',
    20.31 : 'Richard_Tabano',
    33.31 : 'Rommel_Tupig',
    25.31 : 'Philip_Uy',
    25.56 : 'Jayson_Valenzuela',
    22.06 : 'Aris_Jastin_Venan',
    23.56 : 'Angelica_Villanueva',
    22.46 : 'Jared_Sy',
    26.83 : 'Nyles_Chan',
    29.76 : 'Paul_Delariarte',
}

filename = list_filenames[user_specified_layout]
cols_all = list_columns[user_specified_layout + '_all']
cols_cat_inp = list_columns[user_specified_layout + '_cat_inp']
cols_num_inp = list_columns[user_specified_layout + '_num_inp']
cols_inp = cols_cat_inp + cols_num_inp
cols_non_inp = list(set(cols_all) - set(cols_inp))

cols_num_inp_std = ['height','weight','bmi']
cols_num_inp_nrm = list(set(cols_num_inp) - set(cols_num_inp_std))

col_grp = 'full_name'
col_trg = 'posture_id'

pd.set_option('display.float_format', lambda x: '{:.2f}'.format(x))
sns.reset_orig()

## Step 2b. Declare and prepare needed variables

In [ ]:
df = pd.read_csv(filename, usecols = cols_all)

df = df[df.nth <= 5]

if(isinstance(user_specified_nth, int)):
    print('user_specified_nth is an instance of int')
    df = df[df.nth == user_specified_nth]

In [ ]:
def categorize(row):
    if row['posture_label'] == 'Yearner_Right' or row['posture_label'] == 'Log_Right':
        return 100
    elif row['posture_label'] == 'Yearner_Left' or row['posture_label'] == 'Log_Left':
        return 200
    else:
        return row['posture_id']

df_inp = df[cols_inp]

X = df_inp

y = df[col_trg].values
y_intermediate = df.apply(lambda row: categorize(row), axis=1).values
groups = df[col_grp].values

# TODO: delete cols_identifying_idx and other calls to this variable
cols_identifying_idx = df_inp.columns.get_indexer(['height','weight','bmi'])

In [ ]:
cols_cat_inp_idx = df_inp.columns.get_indexer(cols_cat_inp)
cols_num_inp_idx = df_inp.columns.get_indexer(cols_num_inp)

In [ ]:
steps_cat_inp = [
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
]
pipe_cat_inp = Pipeline(steps_cat_inp)

steps_num_inp = [
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', MinMaxScaler())
]
pipe_num_inp = Pipeline(steps_num_inp)

ct = ColumnTransformer(transformers=[
          ('categorical', pipe_cat_inp, cols_cat_inp_idx),
          ('numerical', pipe_num_inp, cols_num_inp_idx)
])

## Step 2c. Declare needed functions

In [ ]:
def save_incorrect_predictions(actual_targets, predicted_targets, classifier):
    cols_display = ['timestamp','height','weight','bmi','bmi_label','full_name','nth','round','posture_id','posture_label','actl','pred', 'pred_label']

    df_display = df
    df_display['actl'] = actual_targets
    df_display['pred'] = predicted_targets
    df_display['pred_label'] = predicted_targets

    df_incorrect = df_display[cols_display][actual_targets != predicted_targets]
    df_incorrect = df_incorrect.replace({'pred_label': dict_positions})
    df_incorrect.to_csv("incorrect_L{}_{}.csv".format(user_specified_layout_int, classifier), index=False, encoding='utf-8-sig')
    display(df_incorrect)

In [ ]:
def print_metrics(actual_targets, predicted_targets, target_classes, result_identifier):
    print(result_identifier)
    print(accuracy_score(actual_targets, predicted_targets))
    # print(precision_score(actual_targets, predicted_targets, average='micro'))
    # print(recall_score(actual_targets, predicted_targets, average='micro'))
    # print(f1_score(actual_targets, predicted_targets_intermediate, average='micro'))
    # try:
    #     print(classification_report(actual_targets, predicted_targets, target_names=target_classes))
    # except:
    #     print(classification_report(actual_targets, predicted_targets))

In [ ]:
def generate_confusion_matrix(actual_targets, predicted_targets, normalize=None, title='Confusion Matrix'):
    adjusted_target_classes = [dict_positions[x] for x in sorted(set(actual_targets).union(predicted_targets))]
    # adjusted_target_classes = []
    # for i, val in enumerate(sorted(set(actual_targets).union(predicted_targets))):
    #     adjusted_target_classes.append(dict_positions[val])

    """this if condition was put in place for failure analysis"""
    if not (actual_targets == predicted_targets).all():
        if normalize is None:
            # print("Confusion Matrix, Without Normalized Counts")
            ConfusionMatrixDisplay.from_predictions(actual_targets, predicted_targets, 
                                                    display_labels = adjusted_target_classes, 
                                                    xticks_rotation = 45)
        else:
            cnf_matrix = confusion_matrix(actual_targets, predicted_targets)
            cnf_matrix = cnf_matrix.astype('float') / cnf_matrix.sum(axis=1)[:, np.newaxis]
            # print("Confusion Matrix, With Normalized Counts")
            ConfusionMatrixDisplay.from_predictions(actual_targets, predicted_targets, 
                                                    display_labels = adjusted_target_classes, 
                                                    xticks_rotation = 45, 
                                                    normalize=normalize, 
                                                    values_format = '.2f')

        plt.title(title)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')

In [ ]:
def plot_confusion_matrices(actual_targets, predicted_targets, classifier, target_classes, result_identifier):
    np.set_printoptions(precision=2)

    print_metrics(actual_targets, predicted_targets, target_classes, result_identifier)

    # Plot non-normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(actual_targets, predicted_targets, title='{} Confusion Matrix - Layout {} ({}, Without Normalized Counts)'.format(classifier, user_specified_layout_int, result_identifier))
    plt.savefig('{}CM-{}S-L{}-{}-nonnormalized ({} {} results).png'.format(user_specified_output_folder, n_steps, user_specified_layout_int, classifier, result_identifier, user_specified_ntbk_identifier), format='png', bbox_inches="tight")
    plt.show()

    # Plot normalized confusion matrix
    plt.figure()
    generate_confusion_matrix(actual_targets, predicted_targets, normalize='true', title='{} Confusion Matrix - Layout {} ({}, With Normalized Counts)'.format(classifier, user_specified_layout_int, result_identifier))
    plt.savefig('{}CM-{}S-L{}-{}-normalized ({} {} results).png'.format(user_specified_output_folder, n_steps, user_specified_layout_int, classifier, result_identifier, user_specified_ntbk_identifier), format='png', bbox_inches="tight")
    plt.show()

In [ ]:
def generate_pipeline(classifier):
    if classifier == 'MLP':
        pipe = Pipeline([
            ('pre', ct), 
            ('clf', MLPClassifier(random_state=user_random_state))
        ])
    elif classifier == 'SVM':
        pipe = Pipeline([
            ('pre', ct), 
            ('clf', SVC(random_state=user_random_state))
        ])

    return pipe

In [ ]:
def save_final_models(data_x, data_y, data_y_intermediate, classifier):
    pipe_intermediate = generate_pipeline(classifier)
    pipe_intermediate.fit(data_x, data_y_intermediate)
    dump(pipe_intermediate, 'L{}-YearnerLog-Step1.joblib'.format(user_specified_layout_int))

    pipe_rights = generate_pipeline(classifier)
    pipe_rights.fit(data_x[np.isin(data_y, [1,5])], data_y[np.isin(data_y, [1,5])])
    dump(pipe_rights, 'L{}-YearnerLog-Step2rights.joblib'.format(user_specified_layout_int))

    pipe_lefts = generate_pipeline(classifier)
    pipe_lefts.fit(data_x[np.isin(data_y, [2,6])], data_y[np.isin(data_y, [2,6])])
    dump(pipe_lefts, 'L{}-YearnerLog-Step2lefts.joblib'.format(user_specified_layout_int))

In [ ]:
def model(train_x, train_y, test_x, test_y, classifier):
    pipe = generate_pipeline(classifier)

    pipe.fit(train_x, train_y)
    predicted_targets = pipe.predict(test_x)

    return predicted_targets

In [ ]:
def evaluate_subgroup(train_x, train_y, test_x, test_y, intermediate_predicted, list_subgroup_posture_ids, subgroup_id, classifier):
    train_x_subgroup = train_x[np.isin(train_y, list_subgroup_posture_ids)]
    train_y_subgroup = train_y[np.isin(train_y, list_subgroup_posture_ids)]
    test_x_predicted_subgroup = test_x[np.isin(intermediate_predicted, subgroup_id)]
    test_y_corresponding_predicted_subgroup = test_y[np.isin(intermediate_predicted, subgroup_id)]

    predicted_targets_from_subgroup = []
    if test_x_predicted_subgroup.size:
        predicted_targets_from_subgroup = model(train_x_subgroup, train_y_subgroup, 
                                               test_x_predicted_subgroup, test_y_corresponding_predicted_subgroup, 
                                               classifier)
    else:
        print("None of the participant's data was classified as subgroup {}".format(subgroup_id))

    return test_y_corresponding_predicted_subgroup, predicted_targets_from_subgroup

In [ ]:
def save_accuracy_scores(list_actual_targets, list_predicted_targets, info_h, info_w, info_b, classifier):
    cols_scores = ['height','weight','bmi','bmi_label','full_name','intermediate','rights','lefts','overall']

In [ ]:
def evaluate_model(data_x, data_y, data_y_intermediate, classifier):
    group_k_fold = GroupKFold(n_splits=df[col_grp].nunique())

    actual_targets = np.array([])
    actual_targets_intermediate = np.array([])
    predicted_targets_intermediate = np.array([])
    actual_targets_rights = np.array([])
    predicted_targets_rights = np.array([])
    actual_targets_lefts = np.array([])
    predicted_targets_lefts = np.array([])

    """
    simulate k number of folds
    """
    for i, (train_index, test_index) in enumerate(group_k_fold.split(data_x, data_y, groups=groups)):
        train_x, test_x = data_x[train_index], data_x[test_index]
        train_y, test_y = data_y[train_index], data_y[test_index]
        train_y_intermediate, test_y_intermediate = data_y_intermediate[train_index], data_y_intermediate[test_index]
        actual_targets = np.append(actual_targets, test_y)

        info_h = np.unique(test_x[..., cols_identifying_idx[0]])
        info_w = np.unique(test_x[..., cols_identifying_idx[1]])
        info_b = np.unique(test_x[..., cols_identifying_idx[2]])

        print('fold {} (height: {:.2f}, weight: {:.2f}, bmi: {:.2f})'.format(i, info_h[0], info_w[0], info_b[0]))

        """
        intermediate model, group test data into: FR, FL, S, P + YRLR, YLLL
        """

        fold_predicted_targets_intermediate = model(train_x, train_y_intermediate, test_x, test_y_intermediate, classifier)

        actual_targets_intermediate = np.append(actual_targets_intermediate, test_y_intermediate)
        predicted_targets_intermediate = np.append(predicted_targets_intermediate, fold_predicted_targets_intermediate)

        print_metrics(test_y_intermediate, fold_predicted_targets_intermediate, target_classes=list_positions, result_identifier='intermediate')
        """for failure analysis: generate confusion matrix for participants whose data arent perfectly classified by the intermediate model"""
        plt.figure()
        generate_confusion_matrix(test_y_intermediate, fold_predicted_targets_intermediate, normalize='true', title='{} Confusion Matrix (Intermediate) - Layout {} (height: {:.2f}, weight: {:.2f}, bmi: {:.2f})'.format(classifier, user_specified_layout_int, info_h[0], info_w[0], info_b[0]))
        # plt.savefig('{}kfold confusion matrices/CM-{}S-L{}-{}-intermediate-height-{:.2f}, weight-{:.2f}, bmi-{:.2f}.png'.format(user_specified_output_folder, n_steps, user_specified_layout_int, classifier, info_h[0], info_w[0], info_b[0]), format='png', bbox_inches="tight")
        plt.show()

        """
        YRLR subgroup model, sift predicted YRLR from intermediate model into: YR and LR
        """

        subgroup_actual_targets, subgroup_predicted_targets = evaluate_subgroup(train_x, train_y, test_x, test_y, fold_predicted_targets_intermediate, [1,5], 100, classifier)

        print_metrics(subgroup_actual_targets, subgroup_predicted_targets, target_classes=list_positions, result_identifier='right_subgroup')
        # plt.figure()
        # generate_confusion_matrix(subgroup_actual_targets, subgroup_predicted_targets, normalize='true', title='{} Confusion Matrix (Right Subgroup) - Layout {} (height: {:.2f}, weight: {:.2f}, bmi: {:.2f})'.format(classifier, user_specified_layout_int, info_h[0], info_w[0], info_b[0]))
        # # plt.savefig('{}kfold confusion matrices/CM-{}S-L{}-{}-right-subgroup-height-{:.2f}, weight-{:.2f}, bmi-{:.2f}.png'.format(user_specified_output_folder, n_steps, user_specified_layout_int, classifier, info_h[0], info_w[0], info_b[0]), format='png', bbox_inches="tight")
        # plt.show()

        actual_targets_rights = np.append(actual_targets_rights, subgroup_actual_targets)
        predicted_targets_rights = np.append(predicted_targets_rights, subgroup_predicted_targets)

        """
        YLLL subgroup model, sift predicted YLLL from intermediate model into: YL and LL
        """

        subgroup_actual_targets, subgroup_predicted_targets = evaluate_subgroup(train_x, train_y, test_x, test_y, fold_predicted_targets_intermediate, [2,6], 200, classifier)

        print_metrics(subgroup_actual_targets, subgroup_predicted_targets, target_classes=list_positions, result_identifier='left_subgroup')
        # plt.figure()
        # generate_confusion_matrix(subgroup_actual_targets, subgroup_predicted_targets, normalize='true', title='{} Confusion Matrix (Left Subgroup) - Layout {} (height: {:.2f}, weight: {:.2f}, bmi: {:.2f})'.format(classifier, user_specified_layout_int, info_h[0], info_w[0], info_b[0]))
        # # plt.savefig('{}kfold confusion matrices/CM-{}S-L{}-{}-left-subgroup-height-{:.2f}, weight-{:.2f}, bmi-{:.2f}.png'.format(user_specified_output_folder, n_steps, user_specified_layout_int, classifier, info_h[0], info_w[0], info_b[0]), format='png', bbox_inches="tight")
        # plt.show()

        actual_targets_lefts = np.append(actual_targets_lefts, subgroup_actual_targets)
        predicted_targets_lefts = np.append(predicted_targets_lefts, subgroup_predicted_targets)
        
        # save_accuracy_scores(list_actual_targets, list_predicted_targets, info_h, info_w, info_b, classifier)

    plot_confusion_matrices(actual_targets_intermediate, predicted_targets_intermediate, classifier, list_grouped_positions, 'overall_intermediate')
    plot_confusion_matrices(actual_targets_rights, predicted_targets_rights, classifier, list_positions, 'overall_right_subgroup')
    plot_confusion_matrices(actual_targets_lefts, predicted_targets_lefts, classifier, list_positions, 'overall_left_subgroup')

    translated_predicted_targets = []
    ctr_rights = 0
    ctr_lefts = 0
    for target in predicted_targets_intermediate:
        if target == 100:
            translated_predicted_targets.append(predicted_targets_rights[ctr_rights])
            ctr_rights += 1
        elif target == 200:
            translated_predicted_targets.append(predicted_targets_lefts[ctr_lefts])
            ctr_lefts += 1
        else:
            translated_predicted_targets.append(target)

    return actual_targets, translated_predicted_targets

## Step 3. Execute

In [ ]:
X = df_inp.to_numpy()

data = X
target = y

gkf = GroupKFold(n_splits=df[col_grp].nunique())

In [ ]:
def hyperparameter_optimization(classifier):
    if classifier == 'MLP':
        param_grid = [
            {
                'clf__hidden_layer_sizes': [
                    (10,30,10),
                    (50,50,50), (50,100,50), (100,), (50,),
                    (1,),(2,),(3,),(4,),(5,),(6,),(7,),(8,),(9,),(10,),(11,), (12,),(13,),(14,),(15,),(16,),(17,),(18,),(19,),(20,),(21,)
                    ],
                'clf__activation': ['tanh', 'relu'], # ['identity', 'logistic', 'tanh', 'relu'],
                'clf__solver': ['sgd', 'adam'], # ['lbfgs', 'sgd', 'adam'],
                'clf__alpha': [0.0001, 0.05],
                'clf__learning_rate': ['constant', 'adaptive'],
                # 'clf__learning_rate_init': np.arange(0.01, 1),
                'clf__max_iter': [100000],
                # 'clf__early_stoppping': [True],
            }
        ]
    elif classifier == 'SVM':
        param_grid = [
            {
                'clf__C': [0.1, 1, 10, 100], # 0.1 < c < 100; [0.1, 1, 10, 100, 1000]
                'clf__gamma': [0.0001, 0.001, 0.01, 0.1, 1, 'scale', 'auto'], # 0.0001 < gamma < 10
                'clf__kernel': ['rbf', 'poly'],
            }
        ]

    gs = GridSearchCV(generate_pipeline(classifier), param_grid, cv=gkf.split(X, y, groups=groups)).fit(X, y)
    
    # gs = HalvingGridSearchCV(generate_pipeline(classifier), param_grid, 
    #                         #  scoring=['accuracy'],
    #                          n_jobs=-1, 
    #                         # #  refit='acccuracy'???,
    #                          cv=gkf.split(X, y, groups=groups)).fit(X, y)

    # gs = RandomizedSearchCV(generate_pipeline(classifier), param_grid, n_jobs=-1, cv=gkf.split(X, y, groups=groups)).fit(X, y)

    print('The best accuracy score for the training dataset is {:.4f}'.format(gs.best_score_))
    print('The best hyperparameters are {}'.format(gs.best_params_))
    # print('The accuracy score for the testing dataset is {.4f}'.format(grid_search.score(X_test_transformed, y_test)))

    return gs.best_estimator_

In [ ]:
svm_gs = hyperparameter_optimization('SVM')

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(svm_gs, X, y)
display = LearningCurveDisplay(train_sizes=train_sizes, train_scores=train_scores, test_scores=test_scores, score_name="Score")
display.plot()
plt.show()

In [ ]:
mlp_gs = hyperparameter_optimization('MLP')

In [ ]:
train_sizes, train_scores, test_scores = learning_curve(mlp_gs, X, y)
display = LearningCurveDisplay(train_sizes=train_sizes, train_scores=train_scores, test_scores=test_scores, score_name="Score")
display.plot()
plt.show()